# Import Library

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import joblib


# Read Data

In [2]:
df_sentiment=pd.read_csv('labeled_data_final.csv')
df_sentiment.head()
df_sentiment.info

C:\Users\hkhiy\AppData\Local\Temp\ipykernel_5456\2333248143.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sentiment=pd.read_csv('labeled_data_final.csv')


<bound method DataFrame.info of          index reviewer_name reviewer_country       rental_date  \
0            0          Igor   Kraków- Ba Lan  02/12/2023 00:00   
1            1         Kevin     Cannes- Pháp  04/12/2023 00:00   
2            2         Walid     Cannes- Pháp  09/12/2023 00:00   
3            3       Maxence   Vương quốc Anh  11/12/2023 00:00   
4            4          Lisa           Ai/len  10/12/2023 00:00   
...        ...           ...              ...               ...   
470937  533510           é£ž        Việt Nam  25/12/2023 00:00   
470938  533511           é£ž        Việt Nam  21/12/2023 00:00   
470939  533512         Abbie        Việt Nam  25/12/2023 00:00   
470940  533513   Luca Matteo        Việt Nam  24/12/2023 00:00   
470941  533514        Adrian        Việt Nam  26/12/2023 00:00   

                                                 comments language sentiment  
0       best support ever witnessed answer fast kind s...       en  positive  
1    

In [3]:
df_sentiment.dropna(subset=['comments'], inplace=True)
df_sentiment.info

<bound method DataFrame.info of          index reviewer_name reviewer_country       rental_date  \
0            0          Igor   Kraków- Ba Lan  02/12/2023 00:00   
1            1         Kevin     Cannes- Pháp  04/12/2023 00:00   
2            2         Walid     Cannes- Pháp  09/12/2023 00:00   
3            3       Maxence   Vương quốc Anh  11/12/2023 00:00   
4            4          Lisa           Ai/len  10/12/2023 00:00   
...        ...           ...              ...               ...   
470937  533510           é£ž        Việt Nam  25/12/2023 00:00   
470938  533511           é£ž        Việt Nam  21/12/2023 00:00   
470939  533512         Abbie        Việt Nam  25/12/2023 00:00   
470940  533513   Luca Matteo        Việt Nam  24/12/2023 00:00   
470941  533514        Adrian        Việt Nam  26/12/2023 00:00   

                                                 comments language sentiment  
0       best support ever witnessed answer fast kind s...       en  positive  
1    

In [4]:
X=[]
for label in df_sentiment['sentiment']:
    if label=="positive":
        X.append(1)
    if label=="negative":
        X.append(2)
    if label=="neutral":
        X.append(0)
df_sentiment['label']=X
df_sentiment.head()

,index,reviewer_name,reviewer_country,rental_date,comments,language,sentiment,label
0,0,Igor,Kraków- Ba Lan,02/12/2023 00:00,best support ever witnessed answer fast kind s...,en,positive,1
1,1,Kevin,Cannes- Pháp,04/12/2023 00:00,everything ok,en,positive,1
2,2,Walid,Cannes- Pháp,09/12/2023 00:00,great accommodation games owner responded quickly,en,positive,1
3,3,Maxence,Vương quốc Anh,11/12/2023 00:00,thanks nigel tra friendly stay ho chi minh cit...,en,positive,1
4,4,Lisa,Ai/len,10/12/2023 00:00,nigel others dedicated responded quickly stay ...,en,positive,1


# Train and test

In [5]:
# Spliting 80% train & 20% test, random_state=42 --> konsisten
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(df_sentiment['comments'], df_sentiment['label'], test_size = 0.2, random_state = 42)

In [6]:
# Membuat data frame untuk set data train
df_train80 = pd.DataFrame()
df_train80['comments'] = train_X
df_train80['label'] = train_Y

# membuat data frame untuk set data test
df_test20 = pd.DataFrame()
df_test20['comments'] = test_X
df_test20['label'] = test_Y

In [7]:
df_train80

,comments,label
162900,bee father incredibly gracious hospitable woul...,1
255074,amazing stay hosts friendly responded inquirie...,1
322012,really great place getting everything clean ev...,1
358849,great place short stay hcm easy reach anything,1
443836,issues host responsive helpful,1
...,...,...
259178,good price quality ratio,1
365838,rujiret fully service minded much cared lot ma...,1
131932,great host friendly staffs location near good ...,1
146867,hotel room clean convenient wifi spotty proble...,1


In [8]:
df_test20

,comments,label
270870,special stylish spacious location feels like s...,1
6436,enthusiastic hotel staff,1
39283,cute brothers sisters room clean back,1
228709,first night water shower get well sewer well s...,1
44267,hotel beautiful clean friendly hotel staff new...,1
...,...,...
53001,host enthusiastic friendly,1
324988,place advertised excellent unit located away m...,1
118196,nice room friendly holdhouse,1
270967,spent years exploring thailand ba hao one favo...,1


In [9]:
#df_train80.to_csv('df_train80.csv')
#df_test20.to_csv('df_test20.csv')

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect_8020 = TfidfVectorizer()
tfidf_vect_8020.fit(df_sentiment['comments'])

# representasi vektor TF-IDF untuk data train & test
train_X_tfidf_8020 = tfidf_vect_8020.transform(df_train80['comments'])
test_X_tfidf_8020 = tfidf_vect_8020.transform(df_test20['comments'])

In [11]:
print(train_X_tfidf_8020)
#print(test_X_tfidf_8020)

  (0, 100720)	0.17867177059164183
  (0, 72910)	0.1770891602786149
  (0, 51893)	0.27347699865845304
  (0, 42969)	0.3389398246405342
  (0, 40915)	0.15188050192342914
  (0, 40860)	0.33141355338761097
  (0, 39915)	0.20918296622519983
  (0, 38962)	0.16663498640339394
  (0, 37325)	0.126502928292398
  (0, 37104)	0.39748334614272374
  (0, 31602)	0.3958789511401518
  (0, 9366)	0.3581209165005994
  (0, 4494)	0.2892263054746217
  (1, 91565)	0.09890949369074425
  (1, 90543)	0.24361888991033515
  (1, 84691)	0.1549966413192165
  (1, 82829)	0.254662610062199
  (1, 78121)	0.24346285799303624
  (1, 74882)	0.1306469226101975
  (1, 74671)	0.19191292925011774
  (1, 72910)	0.10626798580640372
  (1, 67046)	0.21309157275371118
  (1, 57201)	0.1463568021667944
  (1, 56582)	0.1558960312486955
  (1, 53458)	0.2586646155860335
  :	:
  (376751, 73317)	0.24367322885460285
  (376751, 72419)	0.11666404643501214
  (376751, 69576)	0.1814476933428071
  (376751, 66965)	0.1582570412422848
  (376751, 63183)	0.14920839304510

In [12]:
print(train_X_tfidf_8020.shape)
print(test_X_tfidf_8020.shape)


(376753, 102585)
(94189, 102585)


In [13]:
percent_kept = 0.2  

num_features = int(percent_kept * train_X_tfidf_8020.shape[1])
selector = SelectKBest(chi2, k=num_features)

train_X_tfidf_8020_selected = selector.fit_transform(train_X_tfidf_8020, train_Y)
test_X_tfidf_8020_selected = selector.transform(test_X_tfidf_8020)
     

print(train_X_tfidf_8020_selected.shape)
print(test_X_tfidf_8020_selected.shape)

(376753, 20517)
(94189, 20517)


# Feature Selection

In [14]:
percent_kept = 0.2  
num_features = int(percent_kept * train_X_tfidf_8020.shape[1])

selector = SelectKBest(chi2, k=num_features)

train_X_tfidf_8020_selected = selector.fit_transform(train_X_tfidf_8020, train_Y)
test_X_tfidf_8020_selected = selector.transform(test_X_tfidf_8020)

# Process

In [15]:
model = SVC(C=1, kernel='rbf', gamma="scale")
 
# fit --> train
model.fit(train_X_tfidf_8020_selected, train_Y)

SVC(C=1)

In [16]:

predictions_SVM_8020 = model.predict(test_X_tfidf_8020_selected)

test_prediction_8020 = pd.DataFrame()
test_prediction_8020['comments'] = test_X
test_prediction_8020['label'] = predictions_SVM_8020


SVM_accuracy_8020 = accuracy_score(predictions_SVM_8020, test_Y)*100

SVM_accuracy_8020 = round(SVM_accuracy_8020,1)
     
test_prediction_8020

,comments,label
270870,special stylish spacious location feels like s...,1
6436,enthusiastic hotel staff,1
39283,cute brothers sisters room clean back,1
228709,first night water shower get well sewer well s...,1
44267,hotel beautiful clean friendly hotel staff new...,1
...,...,...
53001,host enthusiastic friendly,1
324988,place advertised excellent unit located away m...,1
118196,nice room friendly holdhouse,1
270967,spent years exploring thailand ba hao one favo...,1


In [17]:

test_prediction_8020.to_csv("test_prediction_8020_final.csv")

In [18]:
SVM_accuracy_8020

97.9

# Accuracy, Precision, Recall, f1-score


In [19]:
from sklearn.metrics import classification_report

print ("\nHere is the classification report:")
print (classification_report(test_Y, predictions_SVM_8020, zero_division='warn'))


Here is the classification report:
              precision    recall  f1-score   support

           0       0.91      0.93      0.92     11227
           1       0.99      0.99      0.99     80050
           2       0.90      0.78      0.84      2912

    accuracy                           0.98     94189
   macro avg       0.93      0.90      0.92     94189
weighted avg       0.98      0.98      0.98     94189



# Save trained model and vector

In [20]:
joblib.dump(model, "model.pkl")
joblib.dump(tfidf_vect_8020, "vectorizer.pkl")

['vectorizer.pkl']

In [23]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, chi2

chi = [0.1, 0.2, 0.3]
c = [0.1, 1, 10, 100]
kernel = ['rbf', 'poly']
gamma = [0.0001, 0.001, 0.1, 1, 'scale', 'auto']

def mulai(h, i, j, k):
    percent_kept = h
    num_features = int(percent_kept * train_X_tfidf_8020.shape[1])
    selector = SelectKBest(chi2, k=num_features)
    train_X_tfidf_8020_selected = selector.fit_transform(train_X_tfidf_8020, train_Y)
    test_X_tfidf_8020_selected = selector.transform(test_X_tfidf_8020)

    print(f"Chi Square = {h}\tParameter: c={i}, kernel={j}, gamma={k}")
    model = SVC(C=i, kernel=j, gamma=k)
    model.fit(train_X_tfidf_8020_selected, train_Y)

    predictions_SVM_8020 = model.predict(test_X_tfidf_8020_selected)
    test_prediction_8020 = pd.DataFrame()
    test_prediction_8020['Reviews'] = test_X
    test_prediction_8020['Label'] = predictions_SVM_8020
    SVM_accuracy_8020 = accuracy_score(predictions_SVM_8020, test_Y) * 100
    SVM_accuracy_8020 = round(SVM_accuracy_8020, 1)

    print(f"Akurasi: {SVM_accuracy_8020}")
    print("Here is the classification report:")
    print(classification_report(test_Y, predictions_SVM_8020, zero_division=0.0), end="\n")
    print("============================================================")

    data.append({'Chi Square': h, 'C': i, 'Kernel': j, 'Gamma': k, 'Accuracy': SVM_accuracy_8020})

for h in chi:
  data = []
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
  for i in c:
    for j in kernel:
      for k in gamma:
        mulai(h, i, j, k)
  df = pd.DataFrame(data)

  # Sort the DataFrame by Accuracy in descending order
  df_sorted = df.sort_values('Accuracy', ascending=False)

  # Count the number of combinations with the same accuracy
  accuracy_counts = df_sorted['Accuracy'].value_counts().reset_index()
  accuracy_counts.columns = ['Accuracy', 'Count']
  accuracy_counts_sorted = accuracy_counts.sort_values('Accuracy', ascending=False)

  print("All Combinations:")
  print(df)
  print("\nWorst Combination:")
  print(df_sorted.tail(1))
  print("\nModerate Combination:")
  print(df_sorted.iloc[len(df_sorted) // 2])
  print("\nBest Combination:")
  print(df_sorted.head(1))
  print("\nAccuracy Counts:")
  print(accuracy_counts_sorted)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Chi Square = 0.1	Parameter: c=0.1, kernel=rbf, gamma=0.0001
Akurasi: 85.0
Here is the classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     11227
           1       0.85      1.00      0.92     80050
           2       0.00      0.00      0.00      2912

    accuracy                           0.85     94189
   macro avg       0.28      0.33      0.31     94189
weighted avg       0.72      0.85      0.78     94189

Chi Square = 0.1	Parameter: c=0.1, kernel=rbf, gamma=0.001
